# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [7]:
# imports

import os
import json
from IPython.display import Markdown, display, update_display
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [3]:
# set up environment

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [4]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [14]:
# Get gpt-4o-mini to answer, with streaming

system_prompt = """
You receive instructions to exaplain what code does and why.
"""

messages = [
    { "role": "system", "content": system_prompt },
    { "role": "user", "content": question }
]


def stream_response(stream):
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)


def get_openai_stream(model, messages):
    stream = openai.chat.completions.create(model = model, messages = messages, stream = True)
    stream_response(stream)

get_openai_stream(MODEL, messages)

What it does (in Python terms)

- It builds a set of author values by iterating over books and taking book.get("author").
- It only includes an author if book.get("author") is truthy (i.e., not None or an empty/false value).
- Then it uses yield from to yield each element of that set one by one to the caller of the generator.

Why this is done

- Deduplication: Using a set removes duplicate authors, so you get each author at most once.
- Filtering missing values: The if book.get("author") ensures you skip books that don’t have an author or have a None/empty author.
- Lazy output: yield from streams the authors as they’re produced, which can be efficient for large inputs or when you don’t want to materialize the whole list.

Notes and caveats

- The order of authors yielded from a set is not guaranteed (sets are unordered). If you need a predictable or sorted order, consider using a list and/or sorting, e.g. yield from sorted({...}).
- If you want to include empty strings or falsy authors, remove the if condition or adjust it.
- This assumes each book supports .get("author"), i.e., is a mapping/dict-like object. If not, you may get AttributeError.

Example

Suppose books = [{"author": "Alice"}, {"author": "Bob"}, {"author": "Alice"}, {"author": ""}, {"title": "X"}].

- The set comprehension yields {"Alice", "Bob"}.
- yield from then yields "Alice" and "Bob" (order may vary).

In [ ]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')


def get_llama_stream(model, messages):
    stream = ollama.chat.completions.create(model = model, messages = messages, stream = True)
    stream_response(stream)

get_llama_stream(MODEL_LLAMA, messages)

**Code Explanation**

This line of code is written in Python. It combines the concept of `yield` and `Generator Expressions`.

```python
yield from {book.get("author") for book in books if book.get("author")}
```

Here's what it does:

* `{book.get("author") for book in books if book.get("author")}` is a generator expression:
	+ `for book in books` iterates over each item (`book`) in the `books` iterable or collection.
	+ `if book.get("author")` filters out items where `book.get("author"` returns `None`, effectively excluding any items that do not have an "author" key with a non-empty value.
* `yield from ...` is used to yield values from multiple generators. In this case, it yields the filtered authors from each matching book.

**Why**

The purpose of this code is likely to extract a list of book authors from the `books` collection while filtering out books without an author.

Here are some additional details:

* This code assumes that `book.get("author")` returns either a value (e.g., a string or another iterable) or `None`, indicating that the book does not have an author.
* The `yield from` syntax allows us to write a more efficient and concise way of extracting values from multiple sources, rather than using explicit loops.
* This code can be used in a context like a web framework or a database interface, where you want to fetch data in chunks (i.e., book by book), without loading everything into memory all at once.

**Example Usage**

```python
# Sample books collection with some incomplete data
books = [
    {"title": "Book 1", "author": "John Doe"},
    {"title": "Book 2",},  # missing author
    {"title": "Book 3", "publisher": "ABC Publisher"},
]

# Get the authors from complete books
authors = [book.get("author") for book in books if book.get("author")]

print(authors)  # Output: ["John Doe"]
```

In this example, the first line of code would achieve the same result:
```python
yield from {book.get("author") for book in books if book.get("author")}
```

: 